In [1]:

from rdkit import Chem
from rdkit.Chem import AllChem
import numpy as np


In [24]:
smiles = "CC(C)[C@H](C(N[C@@H](CC(N)=O)C(NC(C)(C)C(N(c(cc(cc1)C(NC)=O)c1N1C)C1=O)=O)=O)=O)N(C)C([C@@H](Cc1c[nH]c2c1cccc2)NC(CN(C)C([C@@H](C)N)=O)=O)=O"
mol = Chem.MolFromSmiles(smiles)
mol = Chem.AddHs(mol)
print(mol.GetNumAtoms())

xyz_file = "/Users/zaan/zasaeed@g.hmc.edu - Google Drive/My Drive/OMO Lab - Peptide Cyclization - Zaan Saeed/Data/NewPeptideLibrary/Peptide_R1C2/R1C2_Conformations/R1C2_Conformation_1.xyz"

def load_xyz_coords(mol, xyz_path):
    conf = Chem.Conformer(mol.GetNumAtoms())

    with open(xyz_path, 'r') as f:
        lines = f.readlines()
        lines = lines[2:]

        for i, line in enumerate(lines):
            tokens = line.strip().split()
            if len(tokens) < 4:
                continue
            x, y, z = map(float, tokens[1:4])
            conf.SetAtomPosition(i, (x, y, z))

    mol.RemoveAllConformers()
    mol.AddConformer(conf, assignId=True)
    return mol

mol = load_xyz_coords(mol, xyz_file)

for i, atom in enumerate(mol.GetAtoms()):
    positions = mol.GetConformer().GetAtomPosition(i)
    print(atom.GetSymbol(), positions.x, positions.y, positions.z)


psi_residues = mol.GetSubstructMatches(Chem.MolFromSmiles('NCC(=O)N'))# N[C;X4][C;X3](=[O])N

#C(=O)NCC(=O)N (normal peptide)
#C(=O)NCCC(=O)N (weird peptide / 3 angles)


phi_residues = mol.GetSubstructMatches(Chem.MolFromSmiles('C(=O)NCCC(=O)'))
print(psi_residues)
print(phi_residues)

    # convert xyz to mol object, iterate thru the atoms figuring out which ones to calculate dihederal of, and then get coordinates #thru mol commands (.getposition) or sort

116
C 5.056 -2.575 -6.018
C 5.416 -3.115 -4.621
C 4.267 -4.005 -4.106
C 5.782 -1.976 -3.609
C 6.926 -1.034 -4.046
N 8.051 -1.607 -4.497
C 9.365 -1.013 -4.801
C 9.821 0.102 -3.806
C 9.818 -0.285 -2.323
N 8.771 0.078 -1.592
O 10.738 -0.932 -1.838
C 10.449 -2.111 -4.811
N 10.23 -3.212 -4.071
C 11.049 -4.418 -3.905
C 11.67 -4.95 -5.227
C 12.227 -4.037 -2.971
C 10.173 -5.521 -3.223
N 9.205 -6.224 -3.879
C 8.653 -7.453 -3.422
C 8.964 -8.288 -2.324
C 8.213 -9.481 -2.153
C 7.195 -9.821 -3.083
C 6.897 -8.979 -4.175
C 8.522 -10.435 -1.038
N 8.756 -9.892 0.161
C 9.02 -10.688 1.352
O 8.496 -11.649 -1.227
C 7.656 -7.806 -4.308
N 7.599 -6.814 -5.279
C 6.608 -6.786 -6.352
C 8.504 -5.854 -4.993
O 8.553 -4.793 -5.615
O 10.337 -5.697 -2.01
O 11.454 -1.953 -5.498
O 6.805 0.18 -3.893
N 6.014 -2.457 -2.217
C 5.162 -1.88 -1.17
C 6.975 -3.372 -1.92
C 7.164 -3.941 -0.498
C 6.886 -5.463 -0.566
C 7.247 -6.255 0.654
C 8.483 -6.724 0.937
N 8.469 -7.358 2.159
C 7.209 -7.366 2.717
C 6.399 -6.668 1.769
C 5.029 -6.51